Demo ipynb for LDA

Creating training pipeline on different situations (15/01/2024)

- For all games (using 0.1 and 0.5 of the whole dataset for pipeline developments)
- For top 11 genres

- (if possible): per game, focus on large and small (indie) games

Do the same thing for all three model architectures

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

import gensim

import spacy

import nltk

import pyLDAvis
import pickle
from datetime import datetime

In [2]:
# download nltk stopwords
# import nltk
# nltk.download('stopwords')

# download spacy stopwords
# ...

In [4]:
# training conditions

from enum import Enum

class TRAINING_CONDS(Enum):
    ALL_GAMES = 1
    ALL_GAMES_LARGE = 2
    BY_GENRE = 3

# training condition
training_cond = TRAINING_CONDS.BY_GENRE

In [6]:
# load a dataset

GENRES = ['Action', 'Indie', 'Adventure', 'RPG', 'Strategy', 'Simulation', 'Free to Play', 'Causal', 'Massively Multiplayer', 'Racing', 'Sports']

training_genre_id = 2

if training_cond == TRAINING_CONDS.ALL_GAMES:
    dataset_path = Path('../dataset_cleaned_heartless_sampled_for_demo.pkl')
elif training_cond == TRAINING_CONDS.ALL_GAMES_LARGE:
    dataset_path = Path('../dataset_cleaned_heartless_sampled_for_demo_large.pkl')
elif training_cond == TRAINING_CONDS.BY_GENRE:
    dataset_path = Path(f'../../dataset/topic_modelling/top_11_genres/{training_genre_id:02}_{GENRES[training_genre_id]}.pkl')

# dataset_path = Path('../../dataset/topic_modelling/top_10_games/00_Terraria.pkl')

# dataset_path = Path('../dataset_cleaned_heartless_sampled_for_demo.pkl')


dataset = pd.read_pickle(dataset_path)

dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 636492 entries, 20727 to 4891927
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   index         636492 non-null  int64 
 1   app_id        636492 non-null  int64 
 2   app_name      636492 non-null  object
 3   review_text   636492 non-null  object
 4   review_score  636492 non-null  int64 
 5   review_votes  636492 non-null  int64 
 6   genre_id      636492 non-null  object
 7   category_id   636492 non-null  object
dtypes: int64(4), object(4)
memory usage: 43.7+ MB


In [9]:
# data preprocessing

import re

def clean(raw):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
    result = re.sub('&gt;', "", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', '', result)
    result = re.sub('&#62;', '', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", '', result)
    return result

def deEmojify(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', x)
    
def remove_num(texts):
   output = re.sub(r'\d+', '', texts)
   return output

def unify_whitespaces(x):
    cleaned_string = re.sub(' +', ' ', x)
    return cleaned_string

from nltk.corpus import stopwords
from spacy.lang.en import stop_words
stop=set(stopwords.words("english")) | set(stop_words.STOP_WORDS)     # use more stopwords (detail is in eda_2.ipynb)
def remove_stopword(text):
   text=[word.lower() for word in text.split() if word.lower() not in stop]
   return " ".join(text)

# only keep alphabets
def remove_non_alphabets(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    return text

def cleaning(df, review):
    df[review] = df[review].apply(lambda x: clean(x))
    df[review] = df[review].apply(lambda x: deEmojify(x))
    df[review] = df[review].apply(lambda x: remove_num(x))
    df[review] = df[review].apply(lambda x: unify_whitespaces(x))
    df[review] = df[review].apply(lambda x: remove_non_alphabets(x))
    df[review] = df[review].apply(lambda x: remove_stopword(x))
    df[review] = df[review].apply(lambda x: x.lower())

def cleaning_strlist(str_list):
    str_list = list(map(lambda x: clean(x), str_list))
    str_list = list(map(lambda x: deEmojify(x), str_list))
    str_list = list(map(lambda x: remove_num(x), str_list))
    str_list = list(map(lambda x: unify_whitespaces(x), str_list))
    str_list = list(map(lambda x: remove_non_alphabets(x), str_list))
    str_list = list(map(lambda x: remove_stopword(x), str_list))
    str_list = list(map(lambda x: x.lower(), str_list))
    return str_list

In [10]:
# apply data preprocessing

cleaning(dataset, 'review_text')

In [11]:
X = dataset['review_text'].values

In [12]:
X

array(['kq collection working windows download patch http www sierrahelp com files patches kingsquest kqcollectiondbupdate exe set install c program files steam steamapps common king quest collection steam saved kq collection files program patch steam version kq collection run dos box confirm makes game joy run win bit cons run game dosbox shortcuts patch creates steam launcher work game mouse cursor black white colored',
       'solved http steamcommunity com app discussions years wanting play running issue trying load finally able find solution windows x windows ran updates able launch games old bit windows glory scummvm sierra patch able detect set proper game good sierra cd quality video kq kq way play years ago pick long games great great review game google nod sierrahelp patch steam install need limit colors entire system need tweak open new folders program files x steam steamapps common king quest collection kq double click king quest win exe shortcut boom playable',
       'awe

In [ ]:
# nltk.download('averaged_perceptron_tagger')
# t = nltk.word_tokenize(X[0])
# tt = nltk.pos_tag(t)
# tt

In [13]:
# do lemmatization, but not stemming (as part of speech is important in topic modelling)
# use nltk wordnet for lemmatization

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemma = WordNetLemmatizer()

# from https://stackoverflow.com/questions/25534214/nltk-wordnet-lemmatizer-shouldnt-it-lemmatize-all-inflections-of-a-word

# from: https://www.cnblogs.com/jclian91/p/9898511.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None     # if none -> created as noun by wordnet
    
def lemmatization(text):
   # use nltk to get PoS tag
    tagged = nltk.pos_tag(nltk.word_tokenize(text))

    # then we only need adj, adv, verb, noun
    # convert from nltk Penn Treebank tag to wordnet tag
    wn_tagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1])), tagged))

    # lemmatize by the PoS
    lemmatized = list(map(lambda x: lemma.lemmatize(x[0], pos=x[1] if x[1] else wordnet.NOUN), wn_tagged))
    # lemma.lemmatize(wn_tagged[0], pos=wordnet.NOUN)

    return lemmatized

In [14]:
# lemmatize the data

X_lemmatized = list(map(lambda x: lemmatization(x), X))

In [15]:
X_lemmatized[0]

['kq',
 'collection',
 'work',
 'window',
 'download',
 'patch',
 'http',
 'www',
 'sierrahelp',
 'com',
 'file',
 'patch',
 'kingsquest',
 'kqcollectiondbupdate',
 'exe',
 'set',
 'install',
 'c',
 'program',
 'file',
 'steam',
 'steamapps',
 'common',
 'king',
 'quest',
 'collection',
 'steam',
 'save',
 'kq',
 'collection',
 'file',
 'program',
 'patch',
 'steam',
 'version',
 'kq',
 'collection',
 'run',
 'dos',
 'box',
 'confirm',
 'make',
 'game',
 'joy',
 'run',
 'win',
 'bit',
 'con',
 'run',
 'game',
 'dosbox',
 'shortcut',
 'patch',
 'create',
 'steam',
 'launcher',
 'work',
 'game',
 'mouse',
 'cursor',
 'black',
 'white',
 'color']

In [18]:
# save the lematized data, as separate pickle file

X_lemmatized_file = dataset_path.parent.joinpath('cleaned_lemmatized', dataset_path.stem + '_cleaned_lemmatized.pkl')

if not X_lemmatized_file.parent.exists():
    X_lemmatized_file.parent.mkdir()

with open(X_lemmatized_file, "wb") as f:
    pickle.dump(X_lemmatized, f)

In [ ]:
# load the lematized data, as separate pickle file
# for convenient hyperparameter selection

# X_lemmatized_file = dataset_path.parent.joinpath('cleaned_lemmatized', dataset_path.stem + '_cleaned_lemmatized.pkl')

# with open(X_lemmatized_file, "rb") as f:
#     X_lemmatized = pickle.load(f)

# X_lemmatized[0]

In [19]:
# use gensim to build a dictionary and train our LDAModel

id2word = gensim.corpora.Dictionary(X_lemmatized)

corpus = [id2word.doc2bow(text) for text in X_lemmatized]

In [ ]:
# build a grid search for hyperparameter selection

from gensim.models import CoherenceModel

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics

    From: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(f'num_topics: {num_topics}')
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=num_topics, 
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        print(f'coherence: {coherencemodel.get_coherence()}')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
N_TOPICS = 20

# Online LDA, how to effective train LDA models
# https://papers.nips.cc/paper_files/paper/2010/hash/71f6278d140af599e06ad9bf1ba03cb0-Abstract.html

lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                             id2word=id2word,
                                             num_topics=N_TOPICS,         # later can use grid search to find the best number of topics
                                             random_state=42,
                                             chunksize=2048,                # chunk size affects memory consumption, and updating speed (like DL batch_size). https://groups.google.com/g/gensim/c/FE7_FYSconA
                                             passes=10,                     # no. of passes over the whole corpus. If larger chunksize, then the passes should be larger too.
                                            #  alpha='auto',
                                             workers=3)     # workers = no. of cores (physical cores, but not logical threads)

visualize the data

In [ ]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

save model

we need to save the corpora.Dictionary and the LDA model

In [ ]:
# save the LDA multicore model (and the corpora.Dictionary object) automatically

lda_save_folder = Path(f'lda_model_{datetime.now().strftime("%Y%m%d_%H%M%S")}')
if not lda_save_folder.exists():
    lda_save_folder.mkdir()

lda_model.save(str(lda_save_folder.joinpath('lda_model')))     # no need to add file extension

Evaluation

gensim provide functions to calculate, so we don't need to install octis (as the evaluation backend of octis also relies on gensim)

octis seems awesome for simple development, but it installs many packages ;(

In [ ]:
# corpus = lemmatized words (?) (list of list of str)

# create a result object from the LDAMulticore model for octis evaluation
# referencing from https://github.com/MIND-Lab/OCTIS/blob/master/octis/models/LDA.py
# and guideline in README: https://github.com/MIND-Lab/OCTIS/tree/master
result_lda_online = {}
result_lda_online['topic-word-matrix'] = lda_model.get_topics()

top_words = 10
topics_output = []
for topic in result_lda_online["topic-word-matrix"]:
    top_k = np.argsort(topic)[-top_words:]
    top_k_words = list(reversed([id2word[i] for i in top_k]))
    topics_output.append(top_k_words)
result_lda_online["topics"] = topics_output

def _get_topic_document_matrix(lda_model, corpus, num_topics=10):
    """
    Return the topic representation of the
    corpus
    """

    id_corpus = corpus

    doc_topic_tuples = []
    for document in id_corpus:
        doc_topic_tuples.append(
            lda_model.get_document_topics(document, minimum_probability=0))

    topic_document = np.zeros((num_topics, len(doc_topic_tuples)))

    for ndoc in range(len(doc_topic_tuples)):
        document = doc_topic_tuples[ndoc]
        for topic_tuple in document:
            topic_document[topic_tuple[0]][ndoc] = topic_tuple[1]
    return topic_document

result_lda_online['topic-document-matrix'] = _get_topic_document_matrix(lda_model, corpus, num_topics=N_TOPICS)

In [ ]:
lda_model.show_topics(num_topics=N_TOPICS, num_words=10, formatted=True, log=False)

In [ ]:
# setup: get the model's topics in their native ordering...
all_topics = lda_model.print_topics()
# ...then create a empty list per topic to collect the docs:
docs_per_topic = [[] for _ in all_topics]

# now, for every doc...
for doc_id, doc_bow in enumerate(corpus):
    # ...get its topics...
    doc_topics = lda_model.get_document_topics(doc_bow)
    # ...& for each of its topics...
    for topic_id, score in doc_topics:
        # ...add the doc_id & its score to the topic's doc list
        docs_per_topic[topic_id].append((doc_id, score))

In [ ]:
# If you're interested in the top docs per topic, you can further sort each list's pairs by their score

for doc_list in docs_per_topic:
    doc_list.sort(key=lambda id_and_score: id_and_score[1], reverse=True)

In [ ]:
print(docs_per_topic[0][:10])

In [ ]:
# show top 10 documents for each topic, also the name of the game
for topic_id, docs in enumerate(docs_per_topic):
    print(f'Topic {topic_id + 1}:')
    for doc_id, score in docs[:10]:
        print(f'Game: {dataset.iloc[doc_id]["app_name"]}')
        print(f'Doc ID: {doc_id}')
        print(f'Score: {score}')
        print(f'Doc: {dataset.iloc[doc_id]["review_text"]}')
        print()
    print('\n\n\n\n\n')

In [ ]:
dataset.iloc[1655473]

In [ ]:
X[1655473]

In [ ]:
result_lda_online['topic-document-matrix'][0]

In [ ]:
lda_model.get_topics().shape

In [ ]:
np.sum(result_lda_online['topic-document-matrix'], axis=0)

Evaluation

instead of using octis, we use gensim provided CoherenceModel object,  
as octis also uses this module for calculating the coherence scores

In [ ]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model, texts=X_lemmatized, dictionary=id2word, coherence='c_v')
coherence_cv = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_cv)

In [ ]:
# Compute Coherence Score using c_npmi
coherence_model_lda = CoherenceModel(model=lda_model, texts=X_lemmatized, dictionary=id2word, coherence='c_npmi')
coherence_npmi = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_npmi)

INFERENCE

inference test

In [ ]:
# inference test

inference_test = ["well its been fun guys, but that's it, no more updates, that one was the last one, there is no longer going to be anymore content for this game anymore, there is no way to replay it as there won't be any updates, nope, that was it, the last update, nothing more, this game has no new ways to experience it as there is no more content updates, nothing new to freshen up the experience, its such a shame that this game has no replay-ability, once you beat the game there is like no point to playing again, as they said guys 1.2 will be they final update. nothing more after 1.2, there is no chance they will make another final update right? several years and final updates later: alright, thats it, no more updates we wont be getting anymore, thats it, nothing more, no more updates, for real this time... oh god, redigit made another tweet.",
                  "keeps forcing me to play it",
'''I will leave the cat here, so that everybody who passes by can pet it and give it a thumbs up and awards
　　　 　　／＞　　フ
　　　 　　| 　_　 _ l
　 　　 　／` ミ＿xノ
　　 　 /　　　 　 |
　　　 /　 ヽ　　 ﾉ
　 　 │　　|　|　|
　／￣|　　 |　|　|
　| (￣ヽ＿_ヽ_)__)
　＼二つ''']

inference_test = cleaning_strlist(inference_test)

inference_test = list(map(lambda x: lemmatization(x), inference_test))

corpus_test = [id2word.doc2bow(text) for text in inference_test]

test_output = lda_model[corpus_test]

test_output

In [ ]:
inference_test[-1]

In [ ]:
# test inference

corpus_test = [id2word.doc2bow(text) for text in inference_test]

output_test = lda_model[corpus_test]

for i in range(len(output_test)):
    # print(sorted(test_output[i], key=lambda x: x[1], reverse=True))
    print(sorted(output_test[i], key=lambda x: x[1], reverse=True))

load model (both corpora Dictionary and the LDA model)

In [ ]:
del id2word
del lda_model

model_datetime = datetime(2024, 1, 15, 0, 21, 57)
lda_save_folder = Path(f'lda_model_{model_datetime.strftime("%Y%m%d_%H%M%S")}')

# id2word_load = gensim.corpora.Dictionary.load('lda_model.id2word')
id2word_l = gensim.corpora.Dictionary.load(str(lda_save_folder.joinpath('lda_model.id2word')))

lda_model_l = gensim.models.ldamulticore.LdaMulticore.load(str(lda_save_folder.joinpath('lda_model')))

In [ ]:
corpus_test2 = [id2word_l.doc2bow(text) for text in inference_test]

output_test2 = lda_model_l[corpus_test2]

for i in range(len(output_test2)):
    print(sorted(output_test2[i], key=lambda x: x[1], reverse=True))